In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# import json
import pprint

import boto3
import os
import time
import subprocess

In [3]:
from data.get_data import get_download_links

links_df = get_download_links()
links_df.head()

,month,link,downloaded,size_in_bytes,comment_count
0,2005-12,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
1,2006-01,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
2,2006-02,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
3,2006-03,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
4,2006-04,https://files.pushshift.io/reddit/comments/RC_...,False,0,0


In [4]:
def calculate_polarity(month):
    sia = SIA()
    
    client = MongoClient()
    db = client['myreddit']
    collection_name = 'comments-' + month
    collection = db[collection_name]
    
    cursor = collection.find({'$and': 
                                   [{'$expr': {'$ne': ['$body', '[deleted]']}},
                                    {'$expr': {'$gt': ['$score', 0]}}]
                           })

    # Calculate sentiment for relevant comments
    for doc in cursor:
        _id = doc['_id']
        sentiment = sia.polarity_scores(doc['body'])['compound']
        comments.update({"_id": _id}, {"$set": {"vader_sentiment": sentiment}})
    
    client.close()

In [5]:
def avg_abs_polarity(month):
    client = MongoClient()
    db = client['myreddit']
    collection_name = 'comments-' + month
    collection = db[collection_name]
    
    cursor = collection.find({'$and': 
                                   [{'$expr': {'$ne': ['$body', '[deleted]']}},
                                    {'$expr': {'$gt': ['$score', 0]}}]
                           })
    
    # Calculate average absolute sentiment for month
    total_abs_polarity = 0
    total_score = 0
    for doc in cursor:
        total_abs_polarity += doc['score']*abs(doc['vader_sentiment'])
        total_score += doc['score']

    avg_abs_polarity = total_abs_polarity/total_score
    print('Total absolute sentiment: {}'.format(total_abs_sent))
    print('Avg absolute sentiment:   {}'.format(avg_abs_sent))
    
    client.close()
    
    return avg_abs_polarity

In [7]:
for idx, row in links_df.loc[20:30].iterrows():
    month = row['month']
    print(month)
#     calculate_sentiment(month)

2007-08
2007-09
2007-10
2007-11
2007-12
2008-01
2008-02
2008-03
2008-04
2008-05
2008-06
